In [58]:
import os
os.getcwd()
# os.chdir('..')
print(f'Current working directory is {os.getcwd()}')

Current working directory is c:\Users\blond\Documents\NssProjects\sleigh_the_model


In [59]:
import pandas as pd
import scripts.features as ft
import scripts.models as md
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor

In [60]:
file = 'data/dataformodel.csv'

In [61]:
months_in_adv = 6

In [62]:
data = ft.load_data(file, months_in_adv)

In [63]:
features_to_lag = [
    'pce',
    'inflation',
    'unemployment_rate',
    'revolving_credit',
    'nonrevolving_credit',
    'total_credit'
]

In [64]:
data_with_lag = ft.add_lag_features(data, months_in_adv, features_to_lag)

In [65]:
data_with_lag = data_with_lag.dropna()

In [66]:
X = data_with_lag.drop(columns=['date', 'pce_future'])
y = data_with_lag['pce_future']

In [67]:
test_size = 60
min_train_size = 60
total_months = len(data_with_lag)
max_splits = (total_months - min_train_size) // test_size
print(max_splits)
n_splits = max_splits

12


In [68]:
time_series_split = TimeSeriesSplit(
    n_splits = n_splits,
    test_size = test_size
)

In [69]:
lg = LinearRegression() 

In [70]:
lr_mse_scores, lr_r2_scores, lr_test_results = md.train_eval_model(X, y, lg, time_series_split, data_with_lag, 'Linear Regression')

*** Linear Regression ***

--- Split 1 ---
  TRAIN: 64 months (1959-07 to 1964-10)
  TEST: 60 months (1964-11 to 1969-10)
Model Performance:
    Mean Squared Error:  94.39
    R-squared:  0.97
    Actual PCE: 636.00
    Predicted PCE: 634.25

--- Split 2 ---
  TRAIN: 124 months (1959-07 to 1969-10)
  TEST: 60 months (1969-11 to 1974-10)
Model Performance:
    Mean Squared Error:  1059.11
    R-squared:  0.91
    Actual PCE: 995.00
    Predicted PCE: 973.12

--- Split 3 ---
  TRAIN: 184 months (1959-07 to 1974-10)
  TEST: 60 months (1974-11 to 1979-10)
Model Performance:
    Mean Squared Error:  1358.61
    R-squared:  0.97
    Actual PCE: 1695.20
    Predicted PCE: 1657.35

--- Split 4 ---
  TRAIN: 244 months (1959-07 to 1979-10)
  TEST: 60 months (1979-11 to 1984-10)
Model Performance:
    Mean Squared Error:  4213.82
    R-squared:  0.94
    Actual PCE: 2659.50
    Predicted PCE: 2742.81

--- Split 5 ---
  TRAIN: 304 months (1959-07 to 1984-10)
  TEST: 60 months (1984-11 to 1989-10)



--- Split 9 ---
  TRAIN: 544 months (1959-07 to 2004-10)
  TEST: 60 months (2004-11 to 2009-10)
Model Performance:
    Mean Squared Error:  41122.04
    R-squared:  0.78
    Actual PCE: 10182.30
    Predicted PCE: 10337.75

--- Split 10 ---
  TRAIN: 604 months (1959-07 to 2009-10)
  TEST: 60 months (2009-11 to 2014-10)
Model Performance:
    Mean Squared Error:  5594.97
    R-squared:  0.98
    Actual PCE: 12209.10
    Predicted PCE: 12357.30

--- Split 11 ---
  TRAIN: 664 months (1959-07 to 2014-10)
  TEST: 60 months (2014-11 to 2019-10)
Model Performance:
    Mean Squared Error:  147349.72
    R-squared:  0.76
    Actual PCE: 12234.40
    Predicted PCE: 14890.99

--- Split 12 ---
  TRAIN: 724 months (1959-07 to 2019-10)
  TEST: 60 months (2019-11 to 2024-10)
Model Performance:
    Mean Squared Error:  650324.55
    R-squared:  0.83
    Actual PCE: 20669.50
    Predicted PCE: 20252.30


From looking at the metrics the Linear Regression model preformed the best of the ones tested

In [71]:
xgbr = xgb.XGBRegressor(
    objective = 'reg:squarederror', 
    n_estimators = 100, 
    learning_rate = 0.1, 
    max_depth = 6, 
    random_state = 42
) 

In [72]:
xgbr_mse_scores, xgbr_r2_scores, xgbr_test_results = md.train_eval_model(X, y, xgbr, time_series_split, data_with_lag, 'XGB Regressor')

*** XGB Regressor ***

--- Split 1 ---
  TRAIN: 64 months (1959-07 to 1964-10)
  TEST: 60 months (1964-11 to 1969-10)


Model Performance:
    Mean Squared Error:  12503.04
    R-squared: -2.52
    Actual PCE: 636.00
    Predicted PCE: 436.81

--- Split 2 ---
  TRAIN: 124 months (1959-07 to 1969-10)
  TEST: 60 months (1969-11 to 1974-10)
Model Performance:
    Mean Squared Error:  39586.77
    R-squared: -2.39
    Actual PCE: 995.00
    Predicted PCE: 635.50

--- Split 3 ---
  TRAIN: 184 months (1959-07 to 1974-10)
  TEST: 60 months (1974-11 to 1979-10)
Model Performance:
    Mean Squared Error:  162397.47
    R-squared: -2.78
    Actual PCE: 1695.20
    Predicted PCE: 993.98

--- Split 4 ---
  TRAIN: 244 months (1959-07 to 1979-10)
  TEST: 60 months (1979-11 to 1984-10)
Model Performance:
    Mean Squared Error:  300809.94
    R-squared: -3.02
    Actual PCE: 2659.50
    Predicted PCE: 1693.25

--- Split 5 ---
  TRAIN: 304 months (1959-07 to 1984-10)
  TEST: 60 months (1984-11 to 1989-10)
Model Performance:
    Mean Squared Error:  397969.21
    R-squared: -2.78
    Actual PCE: 3770.00
    Predicted PC

Looking at metrics XGB Regressor model did not out preform Linear Regression  

In [73]:
rfr = RandomForestRegressor(
    random_state=42
)

In [74]:
rfr_mse_scores, rfr_r2_scores, rfr_test_results = md.train_eval_model(X, y, rfr, time_series_split, data_with_lag, 'Random Forest Regressor')

*** Random Forest Regressor ***

--- Split 1 ---
  TRAIN: 64 months (1959-07 to 1964-10)
  TEST: 60 months (1964-11 to 1969-10)
Model Performance:
    Mean Squared Error:  12859.93
    R-squared: -2.62
    Actual PCE: 636.00
    Predicted PCE: 434.93

--- Split 2 ---
  TRAIN: 124 months (1959-07 to 1969-10)
  TEST: 60 months (1969-11 to 1974-10)
Model Performance:
    Mean Squared Error:  54340.66
    R-squared: -3.65
    Actual PCE: 995.00
    Predicted PCE: 592.78

--- Split 3 ---
  TRAIN: 184 months (1959-07 to 1974-10)
  TEST: 60 months (1974-11 to 1979-10)
Model Performance:
    Mean Squared Error:  166027.99
    R-squared: -2.86
    Actual PCE: 1695.20
    Predicted PCE: 988.88

--- Split 4 ---
  TRAIN: 244 months (1959-07 to 1979-10)
  TEST: 60 months (1979-11 to 1984-10)
Model Performance:
    Mean Squared Error:  350074.55
    R-squared: -3.68
    Actual PCE: 2659.50
    Predicted PCE: 1642.58

--- Split 5 ---
  TRAIN: 304 months (1959-07 to 1984-10)
  TEST: 60 months (1984-11

Looking at metrics Random Forest Regressor model did not out preform Linear Regression  

In [75]:
cat = CatBoostRegressor(
    iterations=100,
    learning_rate=0.1,
    depth=6,
    random_state=42,
    verbose=0
)

In [76]:
cat_mse_scores, cat_r2_scores, cat_test_results = md.train_eval_model(X, y, cat, time_series_split, data_with_lag, 'Cat Boost Regressor')

*** Cat Boost Regressor ***

--- Split 1 ---
  TRAIN: 64 months (1959-07 to 1964-10)
  TEST: 60 months (1964-11 to 1969-10)
Model Performance:
    Mean Squared Error:  13968.62
    R-squared: -2.94
    Actual PCE: 636.00
    Predicted PCE: 429.29

--- Split 2 ---
  TRAIN: 124 months (1959-07 to 1969-10)
  TEST: 60 months (1969-11 to 1974-10)
Model Performance:
    Mean Squared Error:  76457.08
    R-squared: -5.55
    Actual PCE: 995.00
    Predicted PCE: 533.24

--- Split 3 ---
  TRAIN: 184 months (1959-07 to 1974-10)
  TEST: 60 months (1974-11 to 1979-10)
Model Performance:
    Mean Squared Error:  234192.20
    R-squared: -4.44
    Actual PCE: 1695.20
    Predicted PCE: 973.76

--- Split 4 ---
  TRAIN: 244 months (1959-07 to 1979-10)
  TEST: 60 months (1979-11 to 1984-10)
Model Performance:
    Mean Squared Error:  643820.79
    R-squared: -7.61
    Actual PCE: 2659.50
    Predicted PCE: 1502.86

--- Split 5 ---
  TRAIN: 304 months (1959-07 to 1984-10)
  TEST: 60 months (1984-11 to 

Looking at metrics Cat Boost Regressor model did not out preform Linear Regression 

In [77]:
lgbm = lgb.LGBMRegressor(
    objective='regression',
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

Tried lgbm, there was multiple warnings of no further splits with positive gain 

baseline model 

In [78]:
baseline = ft.load_data(file, months_in_adv)

In [79]:
baseline

,date,pce,inflation,unemployment_rate,revolving_credit,nonrevolving_credit,total_credit,pce_future
0,1959-01-01,306.1,29.010,6.0,0.00,48961.16,48961.16,317.8
1,1959-02-01,309.6,29.000,5.9,0.00,49513.71,49513.71,320.2
2,1959-03-01,312.7,28.970,5.6,0.00,50007.73,50007.73,324.2
3,1959-04-01,312.2,28.980,5.2,0.00,50463.43,50463.43,322.8
4,1959-05-01,316.1,29.040,5.1,0.00,51007.24,51007.24,322.9
...,...,...,...,...,...,...,...,...
791,2024-12-01,20408.1,317.603,4.1,1316865.89,3671015.14,4987881.03,NaN
792,2025-01-01,20389.0,319.086,4.0,1323514.25,3673358.72,4996872.97,NaN
793,2025-02-01,20469.3,319.775,4.1,1323945.56,3672057.95,4996003.50,NaN
794,2025-03-01,20621.8,319.615,4.2,1313730.92,3678871.89,4992602.81,NaN


In [80]:
baseline['pce_base_pred'] = baseline['pce'].shift(12)

In [81]:
baseline_df = baseline.dropna(subset=['pce_future', 'pce_base_pred']).copy()

actual_values = baseline_df['pce_future']
predicted_values = baseline_df['pce_base_pred']

print(baseline_df[['date', 'pce', 'pce_future', 'pce_base_pred']].head())

         date    pce  pce_future  pce_base_pred
12 1960-01-01  323.6       331.4          306.1
13 1960-02-01  325.3       331.6          309.6
14 1960-03-01  330.2       333.4          312.7
15 1960-04-01  336.5       335.4          312.2
16 1960-05-01  330.0       335.4          316.1


In [82]:
mse = mean_squared_error(actual_values, predicted_values)
print(f"\nSeasonal Naive Baseline - Mean Squared Error (MSE): {mse:.2f}")

# Calculate R-squared (R2)
r2 = r2_score(actual_values, predicted_values)
print(f"Seasonal Naive Baseline - R-squared (R2): {r2:.2f}")


Seasonal Naive Baseline - Mean Squared Error (MSE): 464192.51
Seasonal Naive Baseline - R-squared (R2): 0.98


trying to tune xgboost, 

In [83]:
xgb_tune = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=150,
    learning_rate=0.05, 
    max_depth=5,   
    subsample=0.7, 
    colsample_bytree=0.7,    
    random_state=42   
)

In [84]:
xgb_tune_mse_scores, xgb_tune_r2_scores, xgb_tune_test_results = md.train_eval_model(X, y, xgb_tune, time_series_split, data_with_lag, 'Tuned XGB Regressor')

*** Tuned XGB Regressor ***

--- Split 1 ---
  TRAIN: 64 months (1959-07 to 1964-10)
  TEST: 60 months (1964-11 to 1969-10)
Model Performance:
    Mean Squared Error:  12940.47
    R-squared: -2.65
    Actual PCE: 636.00
    Predicted PCE: 434.52

--- Split 2 ---
  TRAIN: 124 months (1959-07 to 1969-10)
  TEST: 60 months (1969-11 to 1974-10)
Model Performance:
    Mean Squared Error:  43745.71
    R-squared: -2.75
    Actual PCE: 995.00
    Predicted PCE: 623.06

--- Split 3 ---
  TRAIN: 184 months (1959-07 to 1974-10)
  TEST: 60 months (1974-11 to 1979-10)
Model Performance:
    Mean Squared Error:  166081.39
    R-squared: -2.86
    Actual PCE: 1695.20
    Predicted PCE: 988.72

--- Split 4 ---
  TRAIN: 244 months (1959-07 to 1979-10)
  TEST: 60 months (1979-11 to 1984-10)
Model Performance:
    Mean Squared Error:  300602.79
    R-squared: -3.02
    Actual PCE: 2659.50
    Predicted PCE: 1693.41

--- Split 5 ---
  TRAIN: 304 months (1959-07 to 1984-10)
  TEST: 60 months (1984-11 to 